In [6]:
import random
import string

In [40]:
env= 'cap1234'
min_len = 3
max_len = 10
#환경정보는 3~10 자리수로 구성

def generate_word(length):
    result = ''
    word = ''.join(random.sample(string.ascii_letters + string.digits, k=length))
    return word
# 문자 + 숫자로 이루어진 length 길이의 단어열을 생성

def generate_population(size, min_len, max_len):
    population = [] #세대 배열
    for i in range(size):
        length = i % (max_len - min_len + 1) + min_len #균등하게 mix_len ~ max_len 길이로 지정
        population.append(generate_word(length))
    return population
#생성시킬 1세대 유전자 개수, 문자열의 최소길이, 문자열의 최대길이 만큼 출력. 문자열의 길이는 균등하게 부여됨

pop = generate_population(size=100, min_len=min_len, max_len=max_len)

In [41]:
def fit_test(env,gen):
    score = 0
    if(len(env) != len(gen)):
        return score
    len_score = 1
    score += len_score

    for i in range(len(env)):
        if env[i] == gen[i]:
            score += 1

    return score / (len(env) + len_score) * 100

100.0

In [42]:
def compute_performace(population, password):
    performance_list = []
    for individual in population:
        score = fit_test(password, individual)

        # we can predict length of password
        if score > 0:
            pred_len = len(individual)
        performance_list.append([individual, score])

    population_sorted = sorted(performance_list, key=lambda x: x[1], reverse=True)
    return population_sorted, pred_len

def select_survivors(population_sorted, best_sample, lucky_few, password_len):
    next_generation = []

    for i in range(best_sample):
        if population_sorted[i][1] > 0:
            next_generation.append(population_sorted[i][0])

    lucky_survivors = random.sample(population_sorted, k=lucky_few)
    for l in lucky_survivors:
        next_generation.append(l[0])
    
    # generate new population if next_generation is too small
    while len(next_generation) < best_sample + lucky_few:
        next_generation.append(generate_word(length=password_len))

    random.shuffle(next_generation)
    return next_generation

pop_sorted, pred_len = compute_performace(pop, env)
survivors = select_survivors(pop_sorted, best_sample=20, lucky_few=20, password_len=pred_len)

In [43]:
def create_child(individual1, individual2):
    child = ''
    min_len_ind = min(len(individual1), len(individual2))
    for i in range(min_len_ind):
        if (int(100 * random.random()) < 50):
            child += individual1[i]
        else:
            child += individual2[i]
    return child

# 우리는 그렇게 자비롭지 않기때문에 부모를 내 맘대로 짝지어줄 것이다
def create_children(parents, n_child):
    next_population = []
    for i in range(int(len(parents)/2)):
        for j in range(n_child):
            next_population.append(create_child(parents[i], parents[len(parents) - 1 - i]))
    return next_population

children = create_children(survivors, 5)

In [44]:
def mutate_word(word):
    idx = int(random.random() * len(word))
    if (idx == 0):
        word = random.choice(string.ascii_letters + string.digits) + word[1:]
    else:
        word = word[:idx] + random.choice(string.ascii_letters + string.digits) + word[idx+1:]
    return word

def mutate_population(population, chance_of_mutation):
    for i in range(len(population)):
        if random.random() * 100 < chance_of_mutation:
            population[i] = mutate_word(population[i])
    return population

new_generation = mutate_population(population=children, chance_of_mutation=10)

In [83]:
def generate_env():
    result = ''
    word = ''.join(random.sample(string.ascii_letters + string.digits, k=10))
    env = word
    return env

def random_child():
    n_child = random.randint(1, 7)
    return n_child

def random_luck():
    lucky_few = random.randint(1,10)
    return lucky_few

In [113]:
env = generate_env()
n_generation = 10000 #생성시킬 세대 수
population = 100 #생성시킬 1세대 유전자 수
best_sample = 30 #적합하여 살아남은 유전자의 수
lucky_few = 10 #운좋게 살아남은 놈들
n_child = 5 #낳을 자식의 수
chance_of_mutation = 10 #돌연변이율(%)
cnt = 0

pop = generate_population(size=population, min_len=min_len, max_len=max_len)

for g in range(n_generation):
    if g==0:
        print(env)
    pop_sorted, pred_len = compute_performace(population=pop, password=env)
    if g !=0 and g%100 ==0: # ~세대 마다 일어나는 변화
        env = generate_env()
        print()
        print('Environment change! %s' % env)
        cnt =cnt+1
        print()
    #환경 변화 
    
    if int(pop_sorted[0][1]) == 100:
        gen = g-cnt*100
        print('완벽히 적응하게된차수는', gen,'세대,','유전자는', (pop_sorted[0][0]), '입니다')
        break
    lucky_few = random_luck()

    
    survivors = select_survivors(population_sorted=pop_sorted, best_sample=best_sample, lucky_few=lucky_few, password_len=pred_len)
    
    children = create_children(parents=survivors, n_child=n_child)

    new_generation = mutate_population(population=children, chance_of_mutation=10)
    
    pop = new_generation
    # 세대출력
    print('===== %sth Generation =====' % (g + 1))
    print(pop_sorted[0])

b3rCfSeDRi
===== 1th Generation =====
['bkIK8aVCWO', 18.181818181818183]
===== 2th Generation =====
['b3xK87EzWO', 27.27272727272727]
===== 3th Generation =====
['b3xin7AzWO', 27.27272727272727]
===== 4th Generation =====
['b3Bpl3VyAt', 27.27272727272727]
===== 5th Generation =====
['b3xxlKKMAt', 27.27272727272727]
===== 6th Generation =====
['b3xK83VC49', 27.27272727272727]
===== 7th Generation =====
['b3xC87Kn5T', 36.36363636363637]
===== 8th Generation =====
['b3wC83Kn5O', 36.36363636363637]
===== 9th Generation =====
['b3wCl3KM5T', 36.36363636363637]
===== 10th Generation =====
['b3xCl32n5T', 36.36363636363637]
===== 11th Generation =====
['b3xC8KVy4T', 36.36363636363637]
===== 12th Generation =====
['b3xC8oKC5T', 36.36363636363637]
===== 13th Generation =====
['b3wCPoKC5O', 36.36363636363637]
===== 14th Generation =====
['b3xClKKC5O', 36.36363636363637]
===== 15th Generation =====
['b3xCPKKy5O', 36.36363636363637]
===== 16th Generation =====
['b3xCPKKM5T', 36.36363636363637]
=====

['yHoU2XjLfr', 45.45454545454545]
===== 359th Generation =====
['yEUUAXjLfr', 45.45454545454545]
===== 360th Generation =====
['yHoU2XjAfr', 45.45454545454545]
===== 361th Generation =====
['yHUUAXjL5F', 45.45454545454545]
===== 362th Generation =====
['yEoUAXjAfr', 45.45454545454545]
===== 363th Generation =====
['yHoU2XjAnr', 45.45454545454545]
===== 364th Generation =====
['yHoUAXjLfr', 45.45454545454545]
===== 365th Generation =====
['yHEU2XjLfr', 45.45454545454545]
===== 366th Generation =====
['yHEU2XjAfr', 45.45454545454545]
===== 367th Generation =====
['yHEUAXjLfr', 45.45454545454545]
===== 368th Generation =====
['yHoU2XjLf5', 54.54545454545454]
===== 369th Generation =====
['yHoU2XjAf5', 54.54545454545454]
===== 370th Generation =====
['yHoU2XjAf5', 54.54545454545454]
===== 371th Generation =====
['yHoU2XjAf5', 54.54545454545454]
===== 372th Generation =====
['yHEUAXjLf5', 54.54545454545454]
===== 373th Generation =====
['yHoU2XjLf5', 54.54545454545454]
===== 374th Generatio

===== 671th Generation =====
['r5R0PCfmBM', 63.63636363636363]
===== 672th Generation =====
['r5R0PCfmBM', 63.63636363636363]
===== 673th Generation =====
['r5R0PCKgBM', 63.63636363636363]
===== 674th Generation =====
['r5R0PCfgBM', 72.72727272727273]
===== 675th Generation =====
['r5R0PCfgBM', 72.72727272727273]
===== 676th Generation =====
['r5R0PCfgBM', 72.72727272727273]
===== 677th Generation =====
['r5R0PCfgBM', 72.72727272727273]
===== 678th Generation =====
['r5R0PCfgBM', 72.72727272727273]
===== 679th Generation =====
['r5R0PCfgBM', 72.72727272727273]
===== 680th Generation =====
['r5R0PCfgBM', 72.72727272727273]
===== 681th Generation =====
['r5R0PCfggM', 72.72727272727273]
===== 682th Generation =====
['r5R0PCfgdM', 72.72727272727273]
===== 683th Generation =====
['r5R0PCfgBM', 72.72727272727273]
===== 684th Generation =====
['r5R0PCfgBM', 72.72727272727273]
===== 685th Generation =====
['r5R0PCfgBM', 72.72727272727273]
===== 686th Generation =====
['r5R0PCfgBM', 72.72727272

['K0liUWZXwb', 27.27272727272727]
===== 918th Generation =====
['s0lwUWZ1wb', 27.27272727272727]
===== 919th Generation =====
['q0liUWZ1wb', 27.27272727272727]
===== 920th Generation =====
['q0liUWZXwb', 27.27272727272727]
===== 921th Generation =====
['q0l6UWZ1wb', 27.27272727272727]
===== 922th Generation =====
['q0liUWZ1wb', 27.27272727272727]
===== 923th Generation =====
['q0liUYZXwb', 27.27272727272727]
===== 924th Generation =====
['q0liUWZ1wm', 27.27272727272727]
===== 925th Generation =====
['q0liUWZ1wb', 27.27272727272727]
===== 926th Generation =====
['qbliUYu1wb', 36.36363636363637]
===== 927th Generation =====
['qbliUYu1wb', 36.36363636363637]
===== 928th Generation =====
['q0linYufwb', 36.36363636363637]
===== 929th Generation =====
['qbliRWu1wb', 36.36363636363637]
===== 930th Generation =====
['q0liRYu1wb', 36.36363636363637]
===== 931th Generation =====
['q0lbUWu1w4', 36.36363636363637]
===== 932th Generation =====
['qblmUWu1wd', 45.45454545454545]
===== 933th Generatio

['NVMXtsgyhk', 81.81818181818183]
===== 1166th Generation =====
['NVMXtsgyhk', 81.81818181818183]
===== 1167th Generation =====
['NVMXtsgyfk', 81.81818181818183]
===== 1168th Generation =====
['NVMXtsgyhk', 81.81818181818183]
===== 1169th Generation =====
['NVMXtsgAfk', 81.81818181818183]
===== 1170th Generation =====
['NVMXtsgMhk', 81.81818181818183]
===== 1171th Generation =====
['NVMXtsgMfk', 81.81818181818183]
===== 1172th Generation =====
['NVMXtsglhk', 81.81818181818183]
===== 1173th Generation =====
['NVMXtsglfk', 81.81818181818183]
===== 1174th Generation =====
['NVMXtsgMfk', 81.81818181818183]
===== 1175th Generation =====
['NVMXtsglfk', 81.81818181818183]
===== 1176th Generation =====
['NVMXtsgAfk', 81.81818181818183]
===== 1177th Generation =====
['NVMXtsgMfk', 81.81818181818183]
===== 1178th Generation =====
['NVMXtsgAhk', 81.81818181818183]
===== 1179th Generation =====
['NVMXtsgMfk', 81.81818181818183]
===== 1180th Generation =====
['NVMXtsgMhk', 81.81818181818183]
===== 

...

===== 2401th Generation =====
['QDmIUloHh5', 63.63636363636363]
===== 2402th Generation =====
['QDmIUloHh5', 9.090909090909092]
===== 2403th Generation =====
['QDmIUloHh5', 9.090909090909092]
===== 2404th Generation =====
['QDmIUloHh5', 9.090909090909092]
===== 2405th Generation =====
['QDmIUloHh5', 9.090909090909092]
===== 2406th Generation =====
['QDmIUlfHh5', 9.090909090909092]
===== 2407th Generation =====
['QDmIUloHhe', 9.090909090909092]
===== 2408th Generation =====
['QDmIUloHhe', 9.090909090909092]
===== 2409th Generation =====
['MEmIqloHhw', 9.090909090909092]
===== 2410th Generation =====
['MEmIqloHh5', 9.090909090909092]
===== 2411th Generation =====
['QD7IUloHh5', 9.090909090909092]
===== 2412th Generation =====
['MEmIUloWh5', 9.090909090909092]
===== 2413th Generation =====
['QEmIUloHh5', 9.090909090909092]
===== 2414th Generation =====
['MsmIUloWC5', 9.090909090909092]
===== 2415th Generation =====
['MwmIUlKHh5', 9.090909090909092]
===== 2416th Generation =====
['MEmIUloH

['DzyqXomTtP', 45.45454545454545]
===== 2649th Generation =====
['DzGqXomTtP', 45.45454545454545]
===== 2650th Generation =====
['DzyqXomgtP', 45.45454545454545]
===== 2651th Generation =====
['vzyqXodTtP', 54.54545454545454]
===== 2652th Generation =====
['vzyqXodTtP', 54.54545454545454]
===== 2653th Generation =====
['vzyqXodTtP', 54.54545454545454]
===== 2654th Generation =====
['yzyqXodTtP', 54.54545454545454]
===== 2655th Generation =====
['xzzqXodTtP', 54.54545454545454]
===== 2656th Generation =====
['yzGqXodTtP', 54.54545454545454]
===== 2657th Generation =====
['yzGqXodTtP', 54.54545454545454]
===== 2658th Generation =====
['CzyqXodTtP', 54.54545454545454]
===== 2659th Generation =====
['yzGqXodTtP', 54.54545454545454]
===== 2660th Generation =====
['yzGqXodTtk', 54.54545454545454]
===== 2661th Generation =====
['9zzqXodXmP', 54.54545454545454]
===== 2662th Generation =====
['9zGqXodTtP', 54.54545454545454]
===== 2663th Generation =====
['9zGqXodTt7', 63.63636363636363]
===== 

===== 2850th Generation =====
['YZaHMIJU1l', 63.63636363636363]
===== 2851th Generation =====
['YZaHFIJU1x', 72.72727272727273]
===== 2852th Generation =====
['YZaHFIxU1x', 72.72727272727273]
===== 2853th Generation =====
['YZaHFIJU1x', 72.72727272727273]
===== 2854th Generation =====
['YZaHFIxU1x', 72.72727272727273]
===== 2855th Generation =====
['YZaHMIJU1x', 72.72727272727273]
===== 2856th Generation =====
['YZaHFIxU1x', 72.72727272727273]
===== 2857th Generation =====
['YZaHFIJU1x', 72.72727272727273]
===== 2858th Generation =====
['YZaHMIxU1x', 72.72727272727273]
===== 2859th Generation =====
['YZaHF0JU1x', 81.81818181818183]
===== 2860th Generation =====
['YZaHV0xU1x', 81.81818181818183]
===== 2861th Generation =====
['YZaHV0xU1x', 81.81818181818183]
===== 2862th Generation =====
['YZaHV0xU1x', 81.81818181818183]
===== 2863th Generation =====
['YZaHV0xU1x', 81.81818181818183]
===== 2864th Generation =====
['YZaHF0xU1x', 81.81818181818183]
===== 2865th Generation =====
['YZaHV0JU

===== 3038th Generation =====
['OKPUrwPC5J', 45.45454545454545]
===== 3039th Generation =====
['OKnUrwPC5u', 45.45454545454545]
===== 3040th Generation =====
['OKPUrkPC5u', 45.45454545454545]
===== 3041th Generation =====
['OKPUrwPC5u', 45.45454545454545]
===== 3042th Generation =====
['OKnUrkPC5J', 45.45454545454545]
===== 3043th Generation =====
['OKnUrkPC5J', 45.45454545454545]
===== 3044th Generation =====
['OKPUrHPCrJ', 45.45454545454545]
===== 3045th Generation =====
['OKPUrwPCrJ', 45.45454545454545]
===== 3046th Generation =====
['OKPU0nPC5J', 45.45454545454545]
===== 3047th Generation =====
['OKPUrwPCrJ', 45.45454545454545]
===== 3048th Generation =====
['OKnUUwPCgJ', 45.45454545454545]
===== 3049th Generation =====
['OKPUrwPC5J', 45.45454545454545]
===== 3050th Generation =====
['OKPUrHPCgJ', 45.45454545454545]
===== 3051th Generation =====
['OKPU0wPC5J', 45.45454545454545]
===== 3052th Generation =====
['OKPUrwPC5J', 45.45454545454545]
===== 3053th Generation =====
['OKPUrHPC

===== 3261th Generation =====
['2jAycb0dFe', 54.54545454545454]
===== 3262th Generation =====
['2dAycb0dFe', 54.54545454545454]
===== 3263th Generation =====
['29AycbedFe', 54.54545454545454]
===== 3264th Generation =====
['2jgycbedFe', 54.54545454545454]
===== 3265th Generation =====
['2jAfcbedFe', 63.63636363636363]
===== 3266th Generation =====
['2jAfcbedFe', 63.63636363636363]
===== 3267th Generation =====
['29Afcb0dFe', 63.63636363636363]
===== 3268th Generation =====
['2JAfcb0dFe', 63.63636363636363]
===== 3269th Generation =====
['2VAfcb0dFe', 63.63636363636363]
===== 3270th Generation =====
['29Afcb0dFe', 63.63636363636363]
===== 3271th Generation =====
['2VAfcb0dFe', 63.63636363636363]
===== 3272th Generation =====
['29BfcbedFe', 72.72727272727273]
===== 3273th Generation =====
['2jBfcb0dFe', 72.72727272727273]
===== 3274th Generation =====
['2jBfcbedFe', 72.72727272727273]
===== 3275th Generation =====
['2jBfcb0dFe', 72.72727272727273]
===== 3276th Generation =====
['2jBfcb0d